<a href="https://colab.research.google.com/github/JF11579/Adelphi/blob/main/Foley_Joe_Assignmetn_2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Packages

In [ ]:
library(tidyverse)
install.packages("ISLR")
library(ISLR)
library(lubridate)
install.packages("psycho")
library(psycho)
library(class)
install.packages("gmodels")
library(gmodels)
library(ggplot2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Question 1 : KNN

## Data Prep

In [47]:
head(Smarket,5)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
3,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
4,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
5,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [ ]:
Smarket$Year <- as.factor(Smarket$Year)

## Divide the Smarket into two datasets: 2001 through 2004 will be the training data,

In [48]:
Smarket_train <- Smarket %>% filter(Smarket$Year %in% 2001:2004)

In [49]:
dim(Smarket_train )

[1] 998   9

In [50]:
head(Smarket_train)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
3,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
4,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
5,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
6,2001,0.213,0.614,-0.623,1.032,0.959,1.3491,1.392,Up


## and
2005 will be the test data.

In [51]:
Smarket_test <- filter(Smarket, Year == 2005)

In [52]:
dim(Smarket_test )

[1] 252   9

In [53]:
head(Smarket_test)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2005,-0.134,0.008,-0.007,0.715,-0.431,0.7869,-0.812,Down
2,2005,-0.812,-0.134,0.008,-0.007,0.715,1.5108,-1.167,Down
3,2005,-1.167,-0.812,-0.134,0.008,-0.007,1.7210,-0.363,Down
4,2005,-0.363,-1.167,-0.812,-0.134,0.008,1.7389,0.351,Up
5,2005,0.351,-0.363,-1.167,-0.812,-0.134,1.5691,-0.143,Down
6,2005,-0.143,0.351,-0.363,-1.167,-0.812,1.4779,0.342,Up


Create a subset of the Smarket_train data set which only includes columns 2 to 6 (Lag1,
Lag2, Lag3, Lag4, Lag5, and Volume) and name it Smarket_train_x.

In [54]:
#Smarket_train_x <- Smarket_training  %>% select(2:7)
Smarket_train_x <- Smarket_train  %>% select(2:7)

In [55]:
head(Smarket_train_x)

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.381,-0.192,-2.624,-1.055,5.010,1.1913
2,0.959,0.381,-0.192,-2.624,-1.055,1.2965
3,1.032,0.959,0.381,-0.192,-2.624,1.4112
4,-0.623,1.032,0.959,0.381,-0.192,1.2760
5,0.614,-0.623,1.032,0.959,0.381,1.2057
6,0.213,0.614,-0.623,1.032,0.959,1.3491


scale Smarket_train_x

In [56]:
Smarket_train_z <- Smarket_train_x  %>% mutate_at(c('Lag1' , 'Lag2' , 'Lag3' , 'Lag4' , 'Lag5', 'Volume' ) , 
                  ~(scale(.) %>% as.vector))

In [57]:
head(Smarket_train_z, 5 )

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.3090581,-0.1567574,-2.1275234,-0.8528792,4.0296804,-0.6582433
2,0.7791046,0.3092172,-0.1542732,-2.1256689,-0.8503359,-0.2651441
3,0.8384703,0.7792580,0.3106414,-0.1528043,-2.1127836,0.1634536
4,-0.5074241,0.8386230,0.7796129,0.3120195,-0.1559494,-0.3417461
5,0.4985405,-0.5072549,0.8388429,0.7808993,0.3050975,-0.6044350


## Next on the Smarket_test data subset the cols and scale it

In [58]:
dim(Smarket_test)

[1] 252   9

In [59]:
#Smarket_train_x <- training_data %>% select(2:7)
# Smarket_test_data

Smarket_test_y <- Smarket_test %>% select(2:7)



In [60]:
head(Smarket_test_y)

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.134,0.008,-0.007,0.715,-0.431,0.7869
2,-0.812,-0.134,0.008,-0.007,0.715,1.5108
3,-1.167,-0.812,-0.134,0.008,-0.007,1.7210
4,-0.363,-1.167,-0.812,-0.134,0.008,1.7389
5,0.351,-0.363,-1.167,-0.812,-0.134,1.5691
6,-0.143,0.351,-0.363,-1.167,-0.812,1.4779


## Scaling

In [61]:
Smarket_test_z <- Smarket_test_y %>% mutate_at(c('Lag1' , 'Lag2' , 'Lag3' , 'Lag4' , 'Lag5', 'Volume' ) ,
                   ~(scale(.) %>% as.vector))

In [62]:
head(Smarket_test_z )

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.2305914,-0.01302558,-0.03537916,1.07312920,-0.69918635,-3.2718330
2,-1.2783809,-0.23257694,-0.01218575,-0.04572735,1.07499364,-1.1765712
3,-1.8270022,-1.28085735,-0.23175009,-0.02248241,-0.04277072,-0.5681667
4,-0.5844908,-1.82973573,-1.28009253,-0.24253453,-0.01954847,-0.5163568
5,0.5189335,-0.58664215,-1.82900340,-1.29320592,-0.23938578,-1.0078273
6,-0.2445001,0.51729916,-0.58583626,-1.84333622,-1.28903153,-1.2717973


Create a subset of the Smarket_train data set which only includes the Direction column
and name it Smarket_train_label.

In [63]:
Smarket_train_label <- Smarket_train %>% select(9)

In [64]:
head(Smarket_train_label,3)

,Direction
,<fct>
1,Up
2,Up
3,Down


In [65]:
#convert train lables to a vecctor
Smarket_train_label <- Smarket_train_label[ ,1]

Do the same task for the Smarket_test dataset and
name it Smarket_test_label.

In [66]:
dim(Smarket_test)

[1] 252   9

In [70]:
head(Smarket_test)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2005,-0.134,0.008,-0.007,0.715,-0.431,0.7869,-0.812,Down
2,2005,-0.812,-0.134,0.008,-0.007,0.715,1.5108,-1.167,Down
3,2005,-1.167,-0.812,-0.134,0.008,-0.007,1.7210,-0.363,Down
4,2005,-0.363,-1.167,-0.812,-0.134,0.008,1.7389,0.351,Up
5,2005,0.351,-0.363,-1.167,-0.812,-0.134,1.5691,-0.143,Down
6,2005,-0.143,0.351,-0.363,-1.167,-0.812,1.4779,0.342,Up


In [71]:
Smarket_test_label <- Smarket_test  %>% select(9)

In [72]:
head(Smarket_test_label,3)

,Direction
,<fct>
1,Down
2,Down
3,Down


## KNN where k = 1

## Run knn function, using k = 1, to obtain predictions for the test data.

Repeat the previous two tasks using k = 3 and k = 5. Which k has the highest accuracy?

In [73]:


# load the "class" library
# install.packages("class")
library(class)


In [74]:
Smarket_pred <- knn(train = Smarket_train_z , test = Smarket_test_z,
                cl = Smarket_train_label , k= 1 )

In [ ]:
 Smarket_pred 

In [76]:
length(Smarket_pred)

[1] 252

## Step 4: Evaluating model performance 


Create the cross tabulation of predicted vs. actual

In [77]:
length(Smarket_pred )

[1] 252

In [78]:
dim(Smarket_test_label)

[1] 252   1

In [79]:

Smarket_test_label <- Smarket_test_label[ ,1]


In [80]:
CrossTable(x = Smarket_test_label , 
            y = Smarket_pred,
             prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  252 

 
                   | Smarket_pred 
Smarket_test_label |      Down |        Up | Row Total | 
-------------------|-----------|-----------|-----------|
              Down |        54 |        57 |       111 | 
                   |     0.486 |     0.514 |     0.440 | 
                   |     0.446 |     0.435 |           | 
                   |     0.214 |     0.226 |           | 
-------------------|-----------|-----------|-----------|
                Up |        67 |        74 |       141 | 
                   |     0.475 |     0.525 |     0.560 | 
                   |     0.554 |     0.565 |           | 
                   |     0.266 |     0.294 |           | 
-------------------|-----------|-----------|-----------|
      Column Total |       121 |       131 

## KNN where k = 3

In [81]:
Smarket_pred <- knn(train = Smarket_train_z , test = Smarket_test_z,
                cl = Smarket_train_label , k= 3 )

In [82]:
CrossTable(x = Smarket_test_label , 
            y = Smarket_pred,
             prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  252 

 
                   | Smarket_pred 
Smarket_test_label |      Down |        Up | Row Total | 
-------------------|-----------|-----------|-----------|
              Down |        53 |        58 |       111 | 
                   |     0.477 |     0.523 |     0.440 | 
                   |     0.457 |     0.426 |           | 
                   |     0.210 |     0.230 |           | 
-------------------|-----------|-----------|-----------|
                Up |        63 |        78 |       141 | 
                   |     0.447 |     0.553 |     0.560 | 
                   |     0.543 |     0.574 |           | 
                   |     0.250 |     0.310 |           | 
-------------------|-----------|-----------|-----------|
      Column Total |       116 |       136 

## KNN where k = 5

## where k = 5

In [83]:
Smarket_pred <- knn(train = Smarket_train_z , test = Smarket_test_z,
                cl = Smarket_train_label , k= 5 )

In [84]:
CrossTable(x = Smarket_test_label , 
            y = Smarket_pred,
             prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  252 

 
                   | Smarket_pred 
Smarket_test_label |      Down |        Up | Row Total | 
-------------------|-----------|-----------|-----------|
              Down |        59 |        52 |       111 | 
                   |     0.532 |     0.468 |     0.440 | 
                   |     0.484 |     0.400 |           | 
                   |     0.234 |     0.206 |           | 
-------------------|-----------|-----------|-----------|
                Up |        63 |        78 |       141 | 
                   |     0.447 |     0.553 |     0.560 | 
                   |     0.516 |     0.600 |           | 
                   |     0.250 |     0.310 |           | 
-------------------|-----------|-----------|-----------|
      Column Total |       122 |       130 

In [85]:
CrossTable(x = Smarket_test_label , 
            y = Smarket_pred,
             prop.chisq = TRUE)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  252 

 
                   | Smarket_pred 
Smarket_test_label |      Down |        Up | Row Total | 
-------------------|-----------|-----------|-----------|
              Down |        59 |        52 |       111 | 
                   |     0.515 |     0.484 |           | 
                   |     0.532 |     0.468 |     0.440 | 
                   |     0.484 |     0.400 |           | 
                   |     0.234 |     0.206 |           | 
-------------------|-----------|-----------|-----------|
                Up |        63 |        78 |       141 | 
                   |     0.406 |     0.381 |           | 
                   |     0.447 |     0.553 |     0.560 | 
                   |     0.516 |     0.600 |           | 
              

In [86]:
?CrossTable

# Question 2 : K-Means



Use K-means clustering  to cluster US states by the Violent Crime Rates dataset

In [ ]:
head(USArrests)

In [ ]:
?USArrests

# Arrrests per 100k
# UrbanPop = pct of pop living in urban area
# There is an error for MD in something not rounded: We will ignore this
#

Use the scale function to z-score standardize the USArrests data frames and save the
standardized USArrests on a data frame named USArrests _z.

In [ ]:
'''
Smarket_test_z <- Smarket_test %>% mutate_at(c('Lag1' , 'Lag2' , 'Lag3' , 'Lag4' , 'Lag5', 'Volume' ) ,
                   ~(scale(.) %>% as.vector))
'''

In [ ]:
 USArrests_z <- USArrests %>% mutate_at(c('Murder' , 'Assault' , 'UrbanPop' ,'Rape') ,
              ~(scale(.) %>% as.vector))

In [ ]:
head( USArrests_z)

Run the following code so we all get the same results:

In [ ]:
RNGversion("3.5.2")

In [ ]:
set.seed(1)

## Perform K-means clustering with 5 clusters.

## What states are in each cluster?

In [ ]:
states_clustered <- kmeans(USArrests_z , 5)

In [ ]:
states_clustered

In [ ]:
states_clustered$cluster

In [ ]:
#Smarket_test_label <- Smarket_test_label[ ,1]
str(states_clustered)

In [ ]:
states_clustered

## What are the values of the cluster centers?

In [ ]:
states_clustered$centers

## Plot K-Means Clusters

In [ ]:
clusters_df <- states_clustered

In [ ]:
clusters_df <- as.data.frame( states_clustered$cluster, states_clustered$centers)

In [ ]:
ggplot(data = clusters_df, aes(y = centers)) +
  geom_point()